# Few-Shot Templates



## Overview

LangChain's few-shot prompting 提供一個強大的框架，用於引導語言模型生成高質量的輸出，通過供應精心選擇的示例。此技術最小化了對模型微調的需求，同時確保了在多種使用案例中的準確、上下文感知的結果。

- **Few-shot prompt templates** : 定義提示的結構和格式，通過嵌入示例來引導模型生成一致的輸出。
- **Example selection strategies** : 動態選擇與給定查詢最相關的示例，增強模型的上下文理解能力和回應準確性。
- **Chroma vector store** : 一個強大的工具，用於基於語義相似性存儲和检索示例，使提示構建更具可擴展性和效率。

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [FewShotPromptTemplate](#FewShotPromptTemplate)
- [Dynamic Example Selection with Chroma](#dynamic-example-selection-with-chroma)
- [FewShotChatMessagePromptTemplate](#FewShotChatMessagePromptTemplate)

### References

- [LangChain Documentation: Few-shot prompting](https://python.langchain.com/docs/concepts/few_shot_prompting)
- [How to better prompt when doing SQL question-answering](https://python.langchain.com/docs/how_to/sql_prompting/#few-shot-examples)

---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [2]:
%%capture --no-stderr
%pip install langchain-opentutorial

    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^


In [3]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "langchain-chroma",
    ],
    verbose=False,
    upgrade=False,
)

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "09-FewShot-Prompt-Templates",
    }
)

Environment variables have been set successfully.


## ```FewShotPromptTemplate```

Few-shot prompting 提供一個強大的技術，引導語言模型生成準確且上下文相關的輸出，通過提供精心設計的少量示例。LangChain 的 ```FewShotPromptTemplate``` 簡化了這個過程，允許用戶構建靈活且可重用的提示，用於各種任務，如問題回答、摘要和校正。

1. **Designing Few-Shot Prompts**

   - 定義示例，展示輸出的期望結構和風格。
   - 確保示例覆蓋邊緣案例，以提高模型的理解和性能。

2. **Dynamic Example Selection**

   - 使用語義相似性技術或向量基於搜索選擇與查詢最相關的示例。

3. **Integrating Few-Shot Prompts**

   - 將提示模板與語言模型結合，以生成響應的堅固鏈。

### ```FewShotPromptTemplate``` Example

The ```FewShotPromptTemplate``` 允許您向語言模型提供少量示例，展示輸出的期望結構和格式。通過利用這些示例，模型可以更好地理解上下文並生成更準確的回應。此技術特別適合於如問題回答、摘要或生成結構化輸出等任務。

在下面，我們定義了一些示例，以幫助模型通過將問題分解為中間步驟來更有效地回答問題。我們然後使用 ```FewShotPromptTemplate``` 以基於查詢動態格式化提示。

---

In [7]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    temperature=0,  # Creativity
    model_name="gpt-4o-mini",  # Use a valid model name
)

# User query
question = "What is the capital of United States of America?"

# Query the model
response = llm.predict(question)

# Print the response
print(response)

The capital of the United States of America is Washington, D.C.


In [ ]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# 定義 few-shot 提示的範例
examples = [
    {
        "question": "誰活的更久，賈伯斯還是愛因斯坦？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：賈伯斯幾歲去世？
中間答案：賈伯斯在56歲時去世。
額外問題：愛因斯坦幾歲去世？
中間答案：愛因斯坦在76歲時去世。
最終答案是：愛因斯坦
""",
    },
    {
        "question": "Naver的創辦人何時出生？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：誰是Naver的創辦人？
中間答案：Naver由李海珍創立。
額外問題：李海珍何時出生？
中間答案：李海珍出生於1967年6月22日。
最終答案是：1967年6月22日
""",
    },
    {
        "question": "栗谷李珥的母親出生時，在位的是哪位國王？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：栗谷李珥的母親是誰？
中間答案：栗谷李珥的母親是申師任堂。
額外問題：申師任堂何時出生？
中間答案：申師任堂出生於1504年。
額外問題：1504年時朝鮮的國王是誰？
中間答案：1504年時的朝鮮國王是燕山君。
最終答案是：燕山君
""",
    },
    {
        "question": "《原罪犯》和《寄生上流》的導演來自同一個國家嗎？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：《原罪犯》的導演是誰？
中間答案：《原罪犯》的導演是朴贊郁。
額外問題：朴贊郁來自哪個國家？
中間答案：朴贊郁來自韓國。
額外問題：《寄生上流》的導演是誰？
中間答案：《寄生上流》的導演是奉俊昊。
額外問題：奉俊昊來自哪個國家？
中間答案：奉俊昊來自韓國。
最終答案是：是
""",
    },
]

In [9]:
# Create an example prompt template
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# Print the first formatted example
print(example_prompt.format(**examples[0]))

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein



In [10]:
# Initialize the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Example question
question = "How old was Bill Gates when Google was founded?"

# Generate the final prompt
final_prompt = few_shot_prompt.format(question=question)
print(final_prompt)

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein


Question:
When was the founder of Naver born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967


Question:
Who was the reigning king when Yulgok Yi's mother was born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang bo

In [11]:
# Query the model with the final prompt
response = llm.predict(final_prompt)
print(response)

Does this question require additional questions: Yes.  
Additional Question: When was Google founded?  
Intermediate Answer: Google was founded on September 4, 1998.  
Additional Question: What year was Bill Gates born?  
Intermediate Answer: Bill Gates was born on October 28, 1955.  
Additional Question: How old was Bill Gates on September 4, 1998?  
Intermediate Answer: Bill Gates was 42 years old when Google was founded.  
The final answer is: 42 years old.


## Dynamic Example Selection with ```Chroma```

有時我們需要通過多個思考步驟來評估單個問題。將問題分解為步驟並引導到所需的答案可以帶來更好的質量回應。
```Chroma``` 提供了一種高效的方式來存儲和检索基于語義相似性的示例，使工作流中的動態示例選擇成為可能。

1. **Embedding and Vector Store Initialization**

   - 使用 ```OpenAIEmbeddings``` 來嵌入示例。
   - 將嵌入存儲在 ```Chroma``` 向量存儲中以實現高效检索。

2. **Example Storage**

   - 示例會與其內容和元數據一起存儲。
   - 元數據可以包括問題和答案等詳細信息，這些信息在检索後用於進一步處理。

3. **Similarity Search**

   - 請求向量存儲以检索基于輸入的最相似示例。
   - 啟用上下文感知的動態提示創建。

Import the necessary packages and create a vector store.

In [12]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI

# Initialize embeddings and vector store
embeddings = OpenAIEmbeddings()
chroma = Chroma(persist_directory="example_selector", embedding_function=embeddings)

Create example question-answer processes needed to derive the answer.

In [ ]:
examples = [
    {
        "question": "誰活的更久，賈伯斯還是愛因斯坦？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：賈伯斯幾歲去世？
中間答案：賈伯斯在56歲時去世。
額外問題：愛因斯坦幾歲去世？
中間答案：愛因斯坦在76歲時去世。
最終答案是：愛因斯坦
""",
    },
    {
        "question": "Google的創辦人何時出生？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：誰是Google的創辦人？
中間答案：Google由賴利·佩吉和謝爾蓋·布林共同創立。
額外問題：賴利·佩吉何時出生？
中間答案：賴利·佩吉出生於1973年3月26日。
額外問題：謝爾蓋·布林何時出生？
中間答案：謝爾蓋·布林出生於1973年8月21日。
最終答案是：賴利·佩吉出生於1973年3月26日，謝爾蓋·布林出生於1973年8月21日。
""",
    },
    {
        "question": "唐納·川普的母親出生時，當時的美國總統是誰？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：唐納·川普的母親是誰？
中間答案：唐納·川普的母親是瑪麗·安妮·麥克勞德·川普。
額外問題：瑪麗·安妮·麥克勞德·川普何時出生？
中間答案：瑪麗·安妮·麥克勞德·川普出生於1912年5月10日。
額外問題：1912年時的美國總統是誰？
中間答案：1912年時的美國總統是威廉·霍華·塔虎脫。
最終答案是：威廉·霍華·塔虎脫
""",
    },
    {
        "question": "《原罪犯》和《寄生上流》的導演來自同一個國家嗎？",
        "answer": """這個問題需要額外的提問嗎：是的。
額外問題：《原罪犯》的導演是誰？
中間答案：《原罪犯》的導演是朴贊郁。
額外問題：朴贊郁來自哪個國家？
中間答案：朴贊郁來自韓國。
額外問題：《寄生上流》的導演是誰？
中間答案：《寄生上流》的導演是奉俊昊。
額外問題：奉俊昊來自哪個國家？
中間答案：奉俊昊來自韓國。
最終答案是：是
""",
    },
]

創立一個向量存儲並定義 ```DynamicFewShotLearning``` 模板。

In [14]:
# Add examples to the vector store
texts = [example["question"] for example in examples]
metadatas = [example for example in examples]
chroma.add_texts(texts=texts, metadatas=metadatas)

# Set up Example Selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=chroma,  # Only vectorstore is needed
    k=1  # Number of examples to select
)

# Define Few-Shot Prompt Template
example_prompt_template = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)


讓我們運行它以驗證它是否通過我們希望的過程產生答案。

In [15]:
# Query input and process
query = {"question": "How old was Elon Musk when he made Paypal?"}
formatted_prompt = prompt.format(**query)

# Print the constructed prompt
print("Constructed Prompt:\n")
print(formatted_prompt)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Query the language model with the constructed prompt
response = llm.predict(formatted_prompt)
print("\nResponse:\n")
print(response)

Constructed Prompt:

Question:
When was the founder of Google born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Google?
Intermediate Answer: Google was founded by Larry Page and Sergey Brin.
Additional Question: When was Larry Page born?
Intermediate Answer: Larry Page was born on March 26, 1973.
Additional Question: When was Sergey Brin born?
Intermediate Answer: Sergey Brin was born on August 21, 1973.
The final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.



Question:
How old was Elon Musk when he made Paypal?
Answer:

Response:

Does this question require additional questions: Yes.  
Additional Question: When was PayPal founded?  
Intermediate Answer: PayPal was founded in December 1998.  
Additional Question: When was Elon Musk born?  
Intermediate Answer: Elon Musk was born on June 28, 1971.  
Additional Question: How old was Elon Musk in December 1998?  
Intermediate An

## ```FewShotChatMessagePromptTemplate```

創建提示詞對於每個情況都是複雜且乏味的任務。
```FewShotChatMessagePromptTemplate``` 通過結合相關示例和結構化格式來動態生成聊天基於提示詞，這方法特別有效於總結、會議摘要創建或文檔處理等任務。


In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# 初始化 OpenAI 嵌入
embeddings = OpenAIEmbeddings()

# 初始化向量資料庫
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# 定義 few-shot 提示的範例
examples = [
    {
        "instruction": "您是一位會議記錄專家。請根據給定的資訊撰寫會議記錄",
        "input": "2023年12月25日下午3點，XYZ公司召開了市場策略會議。與會人員包括市場部主管John Smith、數位行銷經理Sarah Johnson和社群媒體經理Mike Wilson。會議主要目的是制定2024年上半年的市場策略，並討論新的社群媒體活動構想。會中，主管John Smith首先簡報了近期市場趨勢，隨後各團隊成員分別就各自領域的策略構想進行了簡報。",
        "answer": """
會議記錄：XYZ公司市場策略會議
日期：2023年12月25日
地點：XYZ公司會議室
與會人員：John Smith（市場部主管）、Sarah Johnson（數位行銷經理）、Mike Wilson（社群媒體經理）

1. 開場
   - 由John Smith主管致開場詞
   - 會議目的：制定2024年上半年市場策略，討論新的社群媒體活動構想

2. 市場趨勢概述（John Smith）
   - John Smith主管報告近期市場趨勢分析
   - 分享消費者行為變化和競爭對手策略的見解

3. 數位行銷策略（Sarah Johnson）
   - Sarah Johnson報告數位行銷策略
   - 著重於線上廣告和SEO優化方法

4. 社群媒體活動（Mike Wilson）
   - Mike Wilson提出新的社群媒體活動構想
   - 說明網紅行銷和內容策略計劃

5. 綜合討論
   - 團隊成員分享並討論各項構想
   - 討論各項策略的預算和資源分配

6. 閉幕
   - 確認下次會議時間
   - 指派Sarah Johnson負責整理並分發會議記錄
""",
    },
    {
        "instruction": "您是一位摘要專家。請根據給定的資訊進行內容摘要",
        "input": "本文件是一份20頁的『永續城市發展策略』報告。報告全面涵蓋了永續城市發展的重要性、當前城市化問題，以及實現城市永續發展的各種策略。報告還介紹了多個國家成功的永續城市發展案例，並總結了這些案例的啟示。",
        "answer": """
文件摘要：永續城市發展策略報告

- 重要性：強調永續城市發展的必要性及其社會、經濟和環境效益
- 當前問題：分析當前城市化進程中的主要問題，如環境污染、資源枯竭和日益加劇的不平等
- 策略：提出實現永續城市發展的各種策略，包括生態友好建築、改善公共交通、提高能源效率和加強社區參與
- 案例研究：介紹全球多個城市的成功永續發展案例，如丹麥哥本哈根和日本橫濱的案例
- 啟示：總結從這些案例中學到的關鍵經驗，強調多面向方法、與當地社區合作的重要性，以及長期規劃的必要性

本報告深入分析了永續城市發展如何在實際中以有效形式實現。
""",
    },
    {
        "instruction": "您是一位句子修正專家。請修正以下句子",
        "input": "我們公司計劃引入新的市場策略。通過這個，與客戶的溝通將變得更加有效。",
        "answer": "本公司期望通過引入新的市場策略，更有效地改善客戶溝通。",
    },
]

In [ ]:
# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

# Example query
query = {
    "instruction": "請撰寫會議記錄",
    "input": "2023年12月26日下午3點，ABC科技公司召開了新移動應用程序項目的每周進度會議。與會人員包括項目經理崔炫洙、主開發者黃基淵和UI/UX設計師金泰龍。會議的主要目的是審查項目的當前進度並制定未來里程碑的計劃。每個團隊成員都提供了各自工作領域的更新，並制定了下周的目標。",
}

# Perform similarity search
query_text = query["instruction"] + " " + query["input"]
results = chroma.similarity_search(query_text, k=1)
print(results)

[Document(metadata={'answer': '\nMeeting Minutes: XYZ Company Marketing Strategy Meeting\nDate: December 25, 2023\nLocation: XYZ Company Conference Room\nAttendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)\n\n1. Opening\n   - Meeting began with opening remarks by Team Leader John Smith.\n   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.\n\n2. Market Trend Overview (John Smith)\n   - Team Leader John Smith presented analysis of recent market trends.\n   - Shared insights on consumer behavior changes and competitor strategies.\n\n3. Digital Marketing Strategy (Sarah Johnson)\n   - Sarah Johnson presented digital marketing strategy.\n   - Focused on online advertising and SEO optimization methods.\n\n4. Social Media Campaign (Mike Wilson)\n   - Mike Wilson proposed ideas for new social media campaigns.\n   - Explained plans for influencer marketing and conte

In [18]:
# Print the most similar example
print(f"Most similar example to the input:\n{query_text}\n")
for result in results:
    print(f'Instruction:\n{result.metadata["instruction"]}')
    print(f'Input:\n{result.metadata["input"]}')
    print(f'Answer:\n{result.metadata["answer"]}')

Most similar example to the input:
Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.

Instruction:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information
Input:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the f

In [19]:
# Create the final prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        (
            "human",
            "{instruction}:\n{input}",
        ),
    ]
)

# Combine the prompt and the best example
best_example = results[0].metadata
filled_prompt = final_prompt.format_messages(**best_example)

# Print the filled prompt
print("\nFilled Prompt:\n")
for message in filled_prompt:
    # Determine message type and extract content
    message_type = type(message).__name__  # e.g., SystemMessage, HumanMessage, AIMessage
    content = message.content
    print(f"{message_type}:\n{content}\n")


Filled Prompt:

SystemMessage:
You are a helpful assistant.

HumanMessage:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader John Smith provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.



In [20]:
# Query the model with the filled prompt
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
formatted_prompt = "\n".join([f"{type(message).__name__}:\n{message.content}" for message in filled_prompt])
response = llm.predict(formatted_prompt)

# Print the model's response
print("\nResponse:\n")
print(response)


Response:

**Meeting Minutes**

**Date:** December 25, 2023  
**Time:** 3:00 PM  
**Location:** XYZ Company Conference Room  

**Attendees:**  
- John Smith, Marketing Team Leader  
- Sarah Johnson, Digital Marketing Manager  
- Mike Wilson, Social Media Manager  

**Agenda:**  
1. Overview of recent market trends  
2. Discussion of marketing strategies for the first half of 2024  
3. Ideas for new social media campaigns  

**Minutes:**  

1. **Call to Order:**  
   The meeting was called to order by John Smith at 3:00 PM.

2. **Overview of Recent Market Trends:**  
   - John Smith provided a brief overview of the current market trends affecting the industry. He highlighted key insights that will inform the marketing strategies for the upcoming year.

3. **Presentations on Strategic Ideas:**  
   - **Sarah Johnson (Digital Marketing Manager):**  
     - Presented her strategic ideas focusing on enhancing digital presence through targeted online advertising and SEO optimization.  
    

### Resolving Similarity Search Issues in Example Selector

當計算相似度時，同時使用 ```instruction``` 和 ```input```。然而，僅基於 ```instruction``` 的搜索無法得到準確的相似度結果。

為了解決這個問題，定義了一個自定義的相似度計算類。

以下是一個錯誤检索結果的示例。


In [21]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)


In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# 初始化 OpenAI 嵌入
embeddings = OpenAIEmbeddings()

# 初始化向量資料庫
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# 定義 few-shot 提示的範例
examples = [
    {
        "instruction": "您是一位會議記錄專家。請根據給定的資訊撰寫會議記錄",
        "input": "2023年12月25日下午3點，XYZ公司召開了市場策略會議。與會人員包括市場部主管金秀珍、數位行銷經理朴智敏和社群媒體經理李俊昊。會議主要目的是制定2024年上半年的市場策略，並討論新的社群媒體活動構想。會中，主管金秀珍首先簡報了近期市場趨勢，隨後各團隊成員分別就各自領域的策略構想進行了簡報。",
        "answer": """
會議記錄：XYZ公司市場策略會議
日期：2023年12月25日
地點：XYZ公司會議室
與會人員：金秀珍（市場部主管）、朴智敏（數位行銷經理）、李俊昊（社群媒體經理）

1. 開場
   - 由金秀珍主管致開場詞
   - 會議目的：制定2024年上半年市場策略，討論新的社群媒體活動構想

2. 市場趨勢概述（金秀珍）
   - 金秀珍主管報告近期市場趨勢分析
   - 分享消費者行為變化和競爭對手策略的見解

3. 數位行銷策略（朴智敏）
   - 朴智敏報告數位行銷策略
   - 著重於線上廣告和SEO優化方法

4. 社群媒體活動（李俊昊）
   - 李俊昊提出新的社群媒體活動構想
   - 說明網紅行銷和內容策略計劃

5. 綜合討論
   - 團隊成員分享並討論各項構想
   - 討論各項策略的預算和資源分配

6. 閉幕
   - 確認下次會議時間
   - 指派朴智敏負責整理並分發會議記錄
""",
    },
    {
        "instruction": "您是一位摘要專家。請根據給定的資訊進行內容摘要",
        "input": "本文件是一份20頁的『永續城市發展策略』報告。報告全面涵蓋了永續城市發展的重要性、當前城市化問題，以及實現城市永續發展的各種策略。報告還介紹了多個國家成功的永續城市發展案例，並總結了這些案例的啟示。",
        "answer": """
文件摘要：永續城市發展策略報告

- 重要性：強調永續城市發展的必要性及其社會、經濟和環境效益
- 當前問題：分析當前城市化進程中的主要問題，如環境污染、資源枯竭和日益加劇的不平等
- 策略：提出實現永續城市發展的各種策略，包括生態友好建築、改善公共交通、提高能源效率和加強社區參與
- 案例研究：介紹全球多個城市的成功永續發展案例，如丹麥哥本哈根和日本橫濱的案例
- 啟示：總結從這些案例中學到的關鍵經驗，強調多面向方法、與當地社區合作的重要性，以及長期規劃的必要性

本報告深入分析了永續城市發展如何在實際中以有效形式實現。
""",
    },
    {
        "instruction": "您是一位句子修正專家。請修正以下句子",
        "input": "我們公司計劃引入新的市場策略。通過這個，與客戶的溝通將變得更加有效。",
        "answer": "本公司期望通過引入新的市場策略，更有效地改善客戶溝通。",
    },
]

In [23]:
# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)


['d6022d36-36bd-4f7d-b60f-fdf3954141d8',
 '4596fe6f-b3c8-4184-9e47-cca4e6e749b6',
 '326ce3b0-8a76-4ad4-b83c-78e9af0728f1']

In [24]:
# Use Chroma for example selection
def select_examples(query):
    query_text = query["instruction"] + " " + query.get("input", "")
    results = chroma.similarity_search(query_text, k=1)
    print("\n[select_examples Output]")
    print(f"Query Text: {query_text}")
    for i, result in enumerate(results, start=1):
        print(f"Result {i}:")
        print(f"Page Content: {result.page_content}")
        print(f"Metadata: {result.metadata}")
    return results

# Example selector using Chroma
def custom_selector(query):
    results = select_examples(query)
    selected_examples = [
        {
            "instruction": result.metadata["instruction"],
            "input": result.metadata["input"],
            "answer": result.metadata["answer"],
        }
        for result in results
    ]
    print("\n[custom_selector Output]")
    for i, example in enumerate(selected_examples, start=1):
        print(f"Selected Example {i}:")
        print(f"Instruction: {example['instruction']}")
        print(f"Input: {example['input']}")
        print(f"Answer: {example['answer']}")
    return selected_examples

# Example query to test
query = {
    "instruction": "Please write the meeting minutes",
    "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
}

# Test the functions
custom_selector(query)



[select_examples Output]
Query Text: Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.
Result 1:
Page Content: You are an expert in writing meeting minutes. Please write meeting minutes based on the given information On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader Kim Su-jin, Digital Marketing Manager Park Ji-min, and Social Media Manager Lee Jun-ho. The main purpose of the meeting was to establish marketing strategies for th

[{'instruction': 'You are an expert in writing meeting minutes. Please write meeting minutes based on the given information',
  'input': "On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader Kim Su-jin, Digital Marketing Manager Park Ji-min, and Social Media Manager Lee Jun-ho. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader Kim Su-jin provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.",
  'answer': '\nMeeting Minutes: XYZ Company Marketing Strategy Meeting\nDate: December 25, 2023\nLocation: XYZ Company Conference Room\nAttendees: Kim Su-jin (Marketing Team Leader), Park Ji-min (Digital Marketing Manager), Lee Jun-ho (Social Media Manager)\n\n1. Opening\n   - Meeting began with opening remarks by Team Lead

In [25]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Define a new example prompt template with the custom selector
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

# Create a prompt template with selected examples
def create_fewshot_prompt(query):
    selected_examples = custom_selector(query)
    fewshot_prompt = ""
    for example in selected_examples:
        fewshot_prompt += example_prompt.format(
            instruction=example["instruction"],
            input=example["input"],
            answer=example["answer"],
        )
    return fewshot_prompt


In [27]:
# Create the final prompt for the chain
def create_final_prompt(query):
    fewshot_prompt = create_fewshot_prompt(query)
    final_prompt = (
        "You are a helpful professional assistant.\n\n" + fewshot_prompt + "\n\n"
        f"Human:\n{query['instruction']}:\n{query['input']}\nAssistant:"
    )
    return final_prompt

# Example queries
queries = [
    {
        "instruction": "Draft meeting minutes for the following discussion",
        "input": "On December 26, 2023, ABC Tech's product development team held their weekly progress meeting regarding the new mobile app project. Present were Project Manager John Davis, Lead Developer Emily Chen, and UI/UX Designer Michael Brown. The team reviewed current project milestones and established next steps. Each team member provided updates on their respective workstreams, and the team set deliverables for the upcoming week.",
    },
    {
        "instruction": "Please provide a summary of the following document",
        "input": "This comprehensive 30-page report titled 'Global Economic Outlook 2023' covers sustainable urban development trends, current urbanization challenges, and strategic approaches to sustainable city planning. The document includes case studies of successful urban development initiatives from various countries and key takeaways from these implementations.",
    },
    {
        "instruction": "Please review and correct these sentences",
        "input": "The company anticipates revenue growth this fiscal year. The new strategic initiatives are showing positive results.",
    },
]

# Process each query and print the responses
for query in queries:
    final_prompt = create_final_prompt(query)
    print("\nFinal Prompt:\n")
    print(final_prompt)
    response = llm.predict(final_prompt)
    print("\nModel Response:\n", response)
    print("\n---\n")


[select_examples Output]
Query Text: Draft meeting minutes for the following discussion On December 26, 2023, ABC Tech's product development team held their weekly progress meeting regarding the new mobile app project. Present were Project Manager John Davis, Lead Developer Emily Chen, and UI/UX Designer Michael Brown. The team reviewed current project milestones and established next steps. Each team member provided updates on their respective workstreams, and the team set deliverables for the upcoming week.
Result 1:
Page Content: You are an expert in writing meeting minutes. Please write meeting minutes based on the given information On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media 